In [1]:
stop_words = [
        "a", "about", "above", "across", "after", "afterwards", "again", "against", "all", "almost", "alone", "along",
        "already", "also", "although", "always", "am", "among", "amongst", "amoungst", "amount", "an", "and", "another",
        "any", "anyhow", "anyone", "anything", "anyway", "anywhere", "are", "around", "as", "at", "back", "be",
        "became", "because", "become", "becomes", "becoming", "been", "before", "beforehand", "behind", "being",
        "below", "beside", "besides", "between", "beyond", "bill", "both", "bottom", "but", "by", "call", "can", "co",
        "con", "could", "cry", "de", "describe", "detail", "do", "done", "down", "due", "during", "each", "eg", "eight",
        "either", "eleven", "else", "elsewhere", "empty", "enough", "etc", "even", "ever", "every", "everyone",
        "everything", "everywhere", "except", "few", "fifteen", "fifty", "fill", "find", "fire", "first", "five", "for",
        "former", "formerly", "forty", "found", "four", "from", "front", "full", "further", "get", "give", "go", "had",
        "has", "have", "he", "hence", "her", "here", "hereafter", "hereby", "herein", "hereupon", "hers", "herself",
        "him", "himself", "his", "how", "however", "hundred", "i", "ie", "if", "in", "inc", "indeed", "interest",
        "into", "is", "it", "its", "itself", "keep", "last", "latter", "latterly", "least", "less", "ltd", "made",
        "many", "may", "me", "meanwhile", "might", "mill", "mine", "more", "moreover", "most", "mostly", "move", "much",
        "must", "my", "myself", "name", "namely", "neither", "nevertheless", "next", "nine", "nobody", "now", "nowhere",
        "of", "off", "often", "on", "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our", "ours",
        "ourselves", "out", "over", "own", "part", "per", "perhaps", "please", "put", "rather", "re", "same", "see",
        "serious", "several", "she", "should", "show", "side", "since", "sincere", "six", "sixty", "so", "some",
        "somehow", "someone", "something", "sometime", "sometimes", "somewhere", "still", "such", "system", "take",
        "ten", "than", "that", "the", "their", "them", "themselves", "then", "thence", "there", "thereafter", "thereby",
        "therefore", "therein", "thereupon", "these", "they", "thick", "thin", "third", "this", "those", "though",
        "three", "through", "throughout", "thru", "thus", "to", "together", "too", "top", "toward", "towards", "twelve",
        "twenty", "two", "un", "under", "until", "up", "upon", "us", "very", "via", "was", "we", "well", "were", "what",
        "whatever", "when", "whence", "whenever", "where", "whereafter", "whereas", "whereby", "wherein", "whereupon",
        "wherever", "whether", "which", "while", "whither", "who", "whoever", "whole", "whom", "whose", "why", "will",
        "with", "within", "without", "would", "yet", "you", "your", "yours", "yourself", "yourselves"
        ]

In [2]:
from torch.utils.data import Dataset, DataLoader
import csv
import json
import re
import os

class Mydataset(Dataset):
    def __init__(self, train=True):
        def clean_str(string):
            string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
            string = re.sub(r"\'s", " \'s", string)
            string = re.sub(r"\'ve", " \'ve", string)
            string = re.sub(r"n\'t", " n\'t", string)
            string = re.sub(r"\'re", " \'re", string)
            string = re.sub(r"\'d", " \'d", string)
            string = re.sub(r"\'ll", " \'ll", string)
            string = re.sub(r",", " , ", string)
            string = re.sub(r"!", " ! ", string)
            string = re.sub(r"\(", " \( ", string)
            string = re.sub(r"\)", " \) ", string)
            string = re.sub(r"\?", " \? ", string)
            string = re.sub(r"\s{2,}", " ", string)
            return string.strip().lower()
        if train:
            print('loading trainig dataset')
            bf = 'train_bodies.csv'
            sf = 'train_stances.csv'
        else:
            print('loading testing dataset')
            bf = 'competition_test_bodies.csv'
            sf = 'competition_test_stances.csv'
            
        with open(os.path.join('fnc-1', sf), 'r', newline='', encoding='utf-8') as myFile:  
            rdr = csv.reader(myFile)
            next(rdr)
            temp = list(rdr)
            self.stances = [[clean_str(a[0]), a[1], a[2]] for a in temp]
            print(len(self.stances), 'stances')

        with open(os.path.join('fnc-1', bf), 'r', newline='', encoding='utf-8') as myFile:  
            rdr = csv.reader(myFile)
            next(rdr)
            temp = list(rdr)
            self.bodies = dict([[a[0], clean_str(a[1])]for a in temp])
            print(len(self.bodies), 'bodies')
            
        self.temp = [a[0] for a in self.stances] + list(self.bodies.values())

        self.len = len(self.stances)
        self.labels = list(sorted(set([t[2] for t in self.stances])))
        
    def __getitem__(self, index):
        return self.stances[index][0], self.bodies[self.stances[index][1]], self.stances[index][2]
    
    def __len__(self):
        return self.len
        
    def get_labels(self):
        return self.labels
    
    def get_label(self, id):
        return self.labels[id]
    
    def get_label_id(self, label):
        return self.labels.index(label)

In [3]:
BATCH_SIZE = 128

train_dataset = Mydataset()
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=BATCH_SIZE, shuffle=True)

test_dataset = Mydataset(train=False)
test_loader = DataLoader(dataset=test_dataset,
                         batch_size=BATCH_SIZE, shuffle=False)

N_LABELS = len(train_dataset.get_labels())

loading trainig dataset
49972 stances
1683 bodies
loading testing dataset
25413 stances
904 bodies


In [4]:
a = list(test_dataset.bodies.values())
a = [len(b) for b in a]

In [5]:
print(max(a), min(a), sum(a)/len(a))
a.index(28)

19661 28 2219.6172566371683


400

In [7]:
a = [len(a[0]) for a in train_dataset.stances]

In [8]:
print(max(a), min(a), sum(a)/len(a))

226 9 69.37060754022252


In [9]:
vocab = set()
for stances in train_dataset.temp:
    vocab.update(stances.split(' '))
vocab = vocab - set(stop_words)
vocab = list(sorted(vocab))

def get_index(word):
    if word in vocab:
        return vocab.index(word) + 1
    return 0

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import word2vec.wordvector as w2v
from torch.autograd import Variable


class CNN_Text(nn.Module):
    
    def __init__(self, args):
        super(CNN_Text, self).__init__()
        self.args = args
        
        C = args.class_num
        Ci = 1
        Co = args.kernel_num
        Ks = args.kernel_sizes
        V = args.embed_num
        D = args.embed_dim
        
        self.embed = nn.Embedding(V, D)
        
#         self.embed, V, D = w2v.get_embedding()

        
        self.embed = nn.Embedding(V, D)
        
        self.convs1 = nn.ModuleList([nn.Conv2d(Ci, Co, (K, D)) for K in Ks])
        
        self.dropout = nn.Dropout(args.dropout)
        
        self.fc1 = nn.Linear(len(Ks)*Co, C)

    def forward(self, x):
        x = self.embed(x)  # (N, W, D)
        
        if self.args.static:
            x = Variable(x)

        x = x.unsqueeze(1)  # (N, Ci, W, D)

        x = [F.relu(conv(x)).squeeze(3) for conv in self.convs1]  # [(N, Co, W), ...]*len(Ks)

        x = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in x]  # [(N, Co), ...]*len(Ks)

        x = torch.cat(x, 1)

        x = self.dropout(x)  # (N, len(Ks)*Co)

        logit = self.fc1(x)  # (N, C)
        
        return logit

F:\settings\anaconda\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [11]:
class args:
    pass

args = args()

args.class_num = 4
args.kernel_num = 2
args.kernel_sizes = [2,3,4,5,6]
args.dropout = 0.5
args.static = True
args.embed_num = len(vocab) + 1
args.embed_dim = 50
args.lr = 0.001
args.epochs = 256
model = CNN_Text(args)

In [12]:
def build_tensor(titles, bodys, labels):
    label_t = torch.tensor([train_dataset.get_label_id(l) for l in labels], dtype=torch.long)
    
    title_i = [[get_index(w) for w in title.split(' ') if w not in stop_words] for title in titles]
    title_size = max([len(a) for a in title_i])
    title_t = torch.zeros(len(title_i), title_size, dtype=torch.long)
    for li, title in enumerate(title_i):
        for co, word in enumerate(title):
            title_t[li][co] = word

    body_i = [[get_index(w) for w in body.split(' ') if w not in stop_words] for body in bodys]
    body_size = max([len(a) for a in body_i])
    body_t = torch.zeros(len(body_i), body_size, dtype=torch.long)
    for li, body in enumerate(body_i):
        for co, word in enumerate(body):
            body_t[li][co] = word
    return title_t, body_t, label_t 

In [ ]:
import sys

args.log_interval = 1
args.test_interval = 2

def train(model, train_loader, test_loader, args):
    optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
    model.train()
    
    steps = 0
    
    for epoch in range(1, args.epochs+1):
        batch = 0
        for title, body, label in train_loader:
            title_t, body_t, label_t = build_tensor(title, body, label)
            optimizer.zero_grad()
            logit = model(torch.cat([title_t, body_t], 1))
            loss = F.cross_entropy(logit, label_t)
            loss.backward()
            optimizer.step()
            steps += 1
            if steps % args.log_interval == 0:
                corrects = (torch.max(logit, 1)[1].view(label_t.size()).data == label_t.data).sum()
                accuracy = 100.0 * corrects/label_t.shape[0]
                print(
                    '\rBatch[{}] - loss: {:.6f}  acc: {:.4f}%({}/{})'.format(steps,
                                                                             loss.data[0], 
                                                                             accuracy,
                                                                             corrects,
                                                                             label_t.shape[0]))
            if steps % args.test_interval == 0:
                acc = eval(test_loader, model, args)
                
def eval(test_loader, model, args):
    model.eval()
    corrects, avg_loss = 0, 0
    for title, body, label in train_loader:
        title_t, body_t, label_t = build_tensor(title, body, label)
        logit = model(torch.cat([title_t, body_t], 1))
        loss = F.cross_entropy(logit, label_t, size_average=False)
        avg_loss += loss.data[0]
        corrects += (torch.max(logit, 1)[1].view(label_t.size()).data == label_t.data).sum()
    size = len(test_loader.dataset)
    avg_loss /= size
    accuracy = 100.0 * corrects/size
    print('\nEvaluation - loss: {:.6f}  acc: {:.4f}%({}/{}) \n'.format(avg_loss, 
                                                                       accuracy, 
                                                                       corrects, 
                                                                       size))
    return accuracy


try:
    train(model, train_loader, test_loader, args)
except KeyboardInterrupt:
    print('\n' + '-' * 89)
    print('Exiting from training early')

begin training


F:\settings\anaconda\lib\site-packages\ipykernel\__main__.py:28: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Batch[1] - loss: 2.384413  acc: 5.0000%(7/128)
Batch[2] - loss: 2.143292  acc: 7.0000%(9/128)


F:\settings\anaconda\lib\site-packages\ipykernel\__main__.py:42: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
